In [ ]:
import pandas as pd
from pyomo.environ import *

# Your input data
schedule = [
    {'teacher': 'Afshari', 'subject': 'Counseling', 'work_load': 3},
    {'teacher': 'Khoshbakht', 'subject': 'Sciences', 'work_load': 6},
    {'teacher': 'Tajik', 'subject': 'Literature', 'work_load': 4},
    {'teacher': 'Naghavi', 'subject': 'Arabic', 'work_load': 6},
    {'teacher': 'Tavakoli', 'subject': 'Religious', 'work_load': 3},
    {'teacher': 'Aghavali', 'subject': 'Math', 'work_load': 8},
    {'teacher': 'Mehrafar', 'subject': 'English', 'work_load': 4},
    {'teacher': 'Saberi', 'subject': 'Social-studies', 'work_load': 4},
    {'teacher': 'Asheri', 'subject': 'Business&Technology', 'work_load': 3},
    {'teacher': 'Elyasi', 'subject': 'Sciences', 'work_load': 3},
    {'teacher': 'Rad', 'subject': 'Literature', 'work_load': 8},
    {'teacher': 'Khanmohammadi', 'subject': 'Math', 'work_load': 4},
    {'teacher': 'Mehrafar', 'subject': 'English', 'work_load': 2},
    {'teacher': 'Avarand', 'subject': 'Social-studies', 'work_load': 2},
]

# Extracting relevant information from the schedule
teachers = set(entry['teacher'] for entry in schedule)
subjects = set(entry['subject'] for entry in schedule)
classes = ['6th', '7th', '8th']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
time_slots = ['8:00 AM - 8:45 AM', '9:10 AM - 9:50 AM', '10:15 AM - 11:00 AM', '11:20 AM - 12:10 PM']


# Create a concrete model
model = ConcreteModel()
# Sets
model.classes = classes
model.days = days
model.time_slots = time_slots
model.teachers = teachers
model.subjects = subjects
model.schedule = schedule
model.work_load = {entry['teacher']: entry['work_load'] for entry in model.schedule}


# Binary variables
model.x = Var(model.teachers,model.subjects, model.classes, model.time_slots, model.days, within=Binary)
model.y = Var(model.teachers, model.time_slots, model.days, within=Binary)
model.u = Var(model.teachers, model.time_slots, model.days, within=Binary)

# Objective function: Minimize windows between classes
model.objective = Objective(expr=sum(model.x[p,i, j, k, t] for p in model.teachers
                                     for i in model.subjects
                                     for j in model.classes
                                     for k in model.time_slots
                                     for t in model.days), sense=minimize)

# Constraints
# Each class should have only 1 lesson per hour
model.one_lesson_per_hour = Constraint(
    expr=sum(model.x[p,i, j, k, t]for p in model.teachers
             for i in model.subjects
            for j in model.classes
            for k in model.time_slots
            for t in model.days) == 1
)

# Each teacher should teach only one class during class hours
model.one_class_per_teacher = Constraint(
    expr=sum(model.y[i, k, t] for i in model.teachers
            for k in model.time_slots
            for t in model.days) <= 1
)

# Each lesson has a specific number of units that must be taught per week based on units
model.units_per_week = ConstraintList()
for p in model.teachers:
    for i in model.subjects:
        model.units_per_week.add(
            sum(model.x[p,i, j, k, t] for j in model.classes
                for k in model.time_slots
                for t in model.days) == sum(entry['work_load'] for entry in model.schedule if entry['subject'] == i)
    )

# Every teacher is either teaching or unemployed at any given time
model.teacher_employment = ConstraintList()
for i in model.teachers:
    model.teacher_employment.add(
        sum(model.y[i, k, t] for k in model.time_slots
            for t in model.days) +
        sum(model.u[i, k, t] for k in model.time_slots
            for t in model.days) == 1
    )


# Solve the model
solver = SolverFactory('glpk')
results =solver.solve(model)

# Display the results using pandas DataFrame
pyomo_results = []

if results.solver.termination_condition == TerminationCondition.optimal:
    # Display the results
    print("Objective Value:", model.objective())
    print("\nSchedule:")
    for teacher in model.teachers:
        for i in model.subjects:
            for j in model.classes:
                for k in model.time_slots:
                    for t in model.days:
                        if model.x[i, j, k, t].value == 1:
#                         print(f"Class {j} - Subject {i} - Time {k} - Day {t}")
                            pyomo_results.append({
                                'Teacher': teacher,
                                'Subject': i,
                                'Class': j,
                                'Time Slot':k,
                                'Day': t,

        })
else:
    print("Solver did not converge to an optimal solution.")
    print(results.solver.termination_condition)



pyomo_df = pd.DataFrame(pyomo_results)

# Display Pyomo results
print("\nPyomo Results:\n")
print(pyomo_df)



Solver did not converge to an optimal solution.
infeasible

Pyomo Results:

Empty DataFrame
Columns: []
Index: []


In [ ]:
def generate_n_neighbor_solutions(initial_solution, n):
    neighbor_solutions = [initial_solution]

    for _ in range(n):
        neighbor_solution = apply_move_within_neighborhood(copy.deepcopy(initial_solution))
        neighbor_solutions.append(neighbor_solution)

    return neighbor_solutions

def variable_neighborhood_search(initial_solution, days, time_slots, max_iterations, n):
    best_timetable = initial_solution
    best_score = evaluate_timetable(best_timetable)
    print(f"\nScore of Initial Timetable is: {best_score}\n")

    current_timetable = initial_solution
    current_score = best_score

    for iteration in range(max_iterations):
        neighbor_solutions = generate_n_neighbor_solutions(current_timetable, n)

        for neighbor_solution in neighbor_solutions:
            new_score = evaluate_timetable(neighbor_solution)

            if new_score > current_score:
                current_timetable = copy.deepcopy(neighbor_solution)
                current_score = new_score

                if new_score > best_score:
                    best_timetable = copy.deepcopy(neighbor_solution)
                    best_score = new_score
                    print(f"\nNew Best Score found: {best_score}\n")

    return best_timetable

# Display VNS results using pandas DataFrame
vns_results = []

for grade, assignments in result_schedule.items():
    for assignment in assignments:
        vns_results.append({
            'Teacher': assignment['teacher'],
            'Subject': assignment['subject'],
            'Class': grade,
            'Time Slot': assignment['time_slot'],
            'Day': assignment['day']
        })

vns_df = pd.DataFrame(vns_results)

# Display VNS results
print("\nVNS Results:\n")
print(vns_df)


VNS Results:

          Teacher              Subject Class            Time Slot        Day
0        Aghavali                 Math   6th    8:00 AM - 8:45 AM    Tuesday
1        Aghavali                 Math   6th    9:10 AM - 9:50 AM     Monday
2        Aghavali                 Math   6th    8:00 AM - 8:45 AM  Wednesday
3        Aghavali                 Math   6th  11:20 AM - 12:10 PM   Saturday
4             Rad           Literature   6th    9:10 AM - 9:50 AM   Saturday
5             Rad           Literature   6th    9:10 AM - 9:50 AM    Tuesday
6             Rad           Literature   6th  11:20 AM - 12:10 PM     Sunday
7             Rad           Literature   6th  10:15 AM - 11:00 AM  Wednesday
8      Khoshbakht             Sciences   6th  11:20 AM - 12:10 PM  Wednesday
9      Khoshbakht             Sciences   6th    9:10 AM - 9:50 AM  Wednesday
10     Khoshbakht             Sciences   6th  10:15 AM - 11:00 AM     Monday
11        Naghavi               Arabic   6th    8:00 AM - 8:4

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


# Plotting the comparison diagram
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(data=pyomo_df, x='Teacher', hue='Day')
plt.title('Pyomo Results')

plt.subplot(1, 2, 2)
sns.countplot(data=vns_df, x='Teacher', hue='Day')
plt.title('VNS Results')

plt.tight_layout()
plt.show()